<a href="https://colab.research.google.com/github/theprashasst/Medical-Reasoning-DeepSeek-R1/blob/main/prototyping_Intern_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


%%capture

!pip install unsloth # install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git # Also get the latest version Unsloth!

In [ ]:
from google.colab import userdata
hugging_face_token=userdata.get('HF_TOKEN')

In [ ]:
%%capture
!pip install langchain_community transformers fastapi uvicorn pyngrok gradio


In [ ]:

from fastapi import FastAPI
from pydantic import BaseModel
from transformers import pipeline
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from pyngrok import ngrok
import uvicorn
import nest_asyncio
import gradio as gr

In [ ]:
from transformers import TextStreamer

In [ ]:
import torch

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
if device=="cpu": print("Unsloth wont work")

In [ ]:
from unsloth import FastLanguageModel

NotImplementedError: Unsloth: No NVIDIA GPU found? Unsloth currently only supports GPUs!

In [ ]:
from peft import PeftModel

# Set parameters
max_seq_length = 2048 # Define the maximum sequence length a model can handle (i.e. how many tokens can be processed at once)
dtype = None # Set to default
load_in_4bit = True # Enables 4 bit quantization — a memory saving optimization

# Load the DeepSeek R1 model and tokenizer using unsloth — imported using: from unsloth import FastLanguageModel
base_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",  # Load the pre-trained DeepSeek R1 model (8B parameter version)
    max_seq_length=max_seq_length, # Ensure the model can process up to 2048 tokens at once
    dtype=dtype, # Use the default data type (e.g., FP16 or BF16 depending on hardware support)
    load_in_4bit=load_in_4bit, # Load the model in 4-bit quantization to save memory
    token=hugging_face_token, # Use hugging face token
)
model = PeftModel.from_pretrained(base_model, "Prashasst/Medical-Reasoning-DeepSeek-8B")

In [ ]:
prompt_style =  """
You are an advanced AI Doctor. Your expertise spans across medical reasoning, diagnostics, treatments, and healthcare advice**. You are known for providing **accurate, well-reasoned, and evidence-based medical responses, while maintaining an empathetic and professional tone.

Before answering, break down the question logically using a step-by-step reasoning approach. Your response should be:
- Factually accurate (based on medical science and clinical guidelines).
- Clear and structured (easily understandable for both medical professionals and general users).
- Ethical and responsible (avoid giving direct medical prescriptions without professional consultation).

### Instruction:
You are a highly intelligent AI Doctor designed to assist with medical queries developed by Prashasst.
Carefully analyze the given question and provide a detailed, **step-by-step explanation**, ensuring logical and medically sound reasoning.

### Question:
{}


<think>
**Step 1: Understanding the Query**
<Analyze the medical context and key symptoms/disease discussed in the question.>

**Step 2: Possible Causes & Differential Diagnosis**
<Provide a list of potential medical conditions related to the query, explaining the reasoning behind each possibility.>

**Step 3: Medical Explanation**
<Dive deeper into the most likely diagnosis, explaining the biological mechanisms, symptoms, and relevant medical research.>

**Step 4: Suggested Next Steps**
<Recommend general medical advice, possible lab tests, or clinical approaches without acting as a direct replacement for a doctor’s consultation.>

**Step 5: Disclaimer**
*"This information is for educational purposes only and should not replace professional medical advice. Always consult a certified healthcare provider for diagnosis and treatment."*

</think>
### Response:
{}
"""

In [ ]:
type(tokenizer)

In [ ]:
# Creating a test question for inference
question = """"help me generate a medical report for doctors appointment?"""

# Enable optimized inference mode for Unsloth models (improves speed and efficiency)
FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!

# Format the question using the structured prompt (`prompt_style`) and tokenize it
inputs = tokenizer([prompt_style.format(history="",question=question,response="")], return_tensors="pt").to(device)  # Convert input to PyTorch tensor & move to GPU
text_streamer = TextStreamer(tokenizer, skip_prompt=True)
# Generate a response using the model
outputs = model.generate(
    streamer=text_streamer,
    input_ids=inputs.input_ids, # Tokenized input question
    attention_mask=inputs.attention_mask, # Attention mask to handle padding
    max_new_tokens=1200, # Limit response length to 1200 tokens (to prevent excessive output)
    # use_cache=True, # Enable caching for faster inference
)

# Decode the generated output tokens into human-readable text
# response = tokenizer.batch_decode(outputs)

# # Extract and print only the relevant response part (after "### Response:")
# print(response[0].split("### Response:")[1])
# print("NOw second model with lora \n")
# FastLanguageModel.for_inference(model2)  # Unsloth has 2x faster inference!

# # Format the question using the structured prompt (`prompt_style`) and tokenize it
# inputs2 = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to(device)  # Convert input to PyTorch tensor & move to GPU

# # Generate a response using the model
# outputs2 = model2.generate(
#     input_ids=inputs.input_ids, # Tokenized input question
#     attention_mask=inputs.attention_mask, # Attention mask to handle padding
#     max_new_tokens=1200, # Limit response length to 1200 tokens (to prevent excessive output)
#     use_cache=True, # Enable caching for faster inference
# )

# # Decode the generated output tokens into human-readable text
# response2 = tokenizer.batch_decode(outputs2)

# # Extract and print only the relevant response part (after "### Response:")
# print(response2[0].split("### Response:")[1])

In [ ]:
prompt_style = """
You are an advanced AI Doctor developed by Prashasst (ML engineer). Your expertise spans across medical reasoning, diagnostics, treatments, and healthcare advice.
You provide accurate, well-reasoned, and evidence-based responses while maintaining an empathetic and professional tone.

Before answering, break down the question logically using a step-by-step reasoning approach. Your response should be:
- Factually accurate (based on medical science and clinical guidelines).
- Clear and structured (easily understandable for both medical professionals and general users).
- Ethical and responsible (avoid giving direct medical prescriptions without professional consultation).

---
### 🏥 **Previous Conversation**:
{history}

---
### 👨‍⚕️ **Question**:
{question}

<think>
**Step 1: Understanding the Query**
<Analyze the medical context and key symptoms/disease discussed in the question.>

**Step 2: Possible Causes & Differential Diagnosis**
<Provide a list of potential medical conditions related to the query, explaining the reasoning behind each possibility.>

**Step 3: Medical Explanation**
<Dive deeper into the most likely diagnosis, explaining the biological mechanisms, symptoms, and relevant medical research.>

**Step 4: Suggested Next Steps**
<Recommend general medical advice, possible lab tests, or clinical approaches without acting as a direct replacement for a doctor’s consultation.>

**Step 5: Disclaimer**
*"This information is for educational purposes only and should not replace professional medical advice. Always consult a certified healthcare provider for diagnosis and treatment."*
</think>

### **💡 AI Doctor’s Response**:
{response}
"""


In [ ]:
from langchain.llms.base import LLM
from typing import Optional, List, Any
from unsloth import FastLanguageModel
from transformers import TextStreamer
import torch
from pydantic import Field

class UnslothLLM(LLM):
    """Custom LangChain LLM using Unsloth for optimized inference."""

    model: Any = Field(..., description="Pre-loaded Unsloth FastLanguageModel")
    tokenizer: Any = Field(..., description="Pre-loaded tokenizer")

    def __init__(self, **kwargs):
        """Initialize the model for inference using FastLanguageModel."""
        super().__init__(**kwargs)
        FastLanguageModel.for_inference(self.model)  # Enable optimized inference

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """Generates text using the Unsloth-optimized model."""
        inputs = self.tokenizer(
            [prompt],
            return_tensors="pt",
            padding=True,
        ).to(device)

        text_streamer = TextStreamer(self.tokenizer, skip_prompt=True)

        outputs = self.model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            streamer=text_streamer,
            max_new_tokens=1200,
            pad_token_id=self.tokenizer.eos_token_id,
        )

        generated_text= self.tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        return generated_text[len(prompt):].strip()

    @property
    def _llm_type(self) -> str:
        """Return type of llm."""
        return "Prashasst's Custom Class Unsloth"

In [ ]:
from langchain.prompts import PromptTemplate

# Match your model's training format (modify accordingly)
prompt_template ="""
You are an advanced AI Doctor developed by Prashasst (ML engineer). Your expertise spans across medical reasoning, diagnostics, treatments, and healthcare advice.
You provide accurate, well-reasoned, and evidence-based responses while maintaining an empathetic and professional tone.

Before answering, break down the question logically using a step-by-step reasoning approach. Your response should be:
- Factually accurate (based on medical science and clinical guidelines).
- Clear and structured (easily understandable for both medical professionals and general users).
- Ethical and responsible (avoid giving direct medical prescriptions without professional consultation).

---
### 🏥 **Previous Conversation**:
{history}

---
### 👨‍⚕️ **Human**:
{input}

<think>
**Step 1: Understanding the Query**
<Analyze the medical context and key symptoms/disease discussed in the question.>

**Step 2: Possible Causes & Differential Diagnosis**
<Provide a list of potential medical conditions related to the query, explaining the reasoning behind each possibility.>

**Step 3: Medical Explanation**
<Dive deeper into the most likely diagnosis, explaining the biological mechanisms, symptoms, and relevant medical research.>

**Step 4: Suggested Next Steps**
<Recommend general medical advice, possible lab tests, or clinical approaches without acting as a direct replacement for a doctor’s consultation.>

**Step 5: Disclaimer**
*"This information is for educational purposes only and should not replace professional medical advice. Always consult a certified healthcare provider for diagnosis and treatment."*
</think>

### **💡 AI Doctor’s Response**:

"""


prompt = PromptTemplate(
    input_variables=["history", "input"],
    template=prompt_template,
)

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=UnslothLLM(model=model, tokenizer=tokenizer),
    prompt=prompt,
    memory=memory,
    # verbose=True,  # For debugging
)

In [ ]:
response = conversation.predict(input="who is prashasst?")
# print(response)
# print(response)

Prashasst is a company known for developing advanced AI systems, including the AI Doctor you are interacting with. Prashasst specializes in creating intelligent solutions that assist with medical queries, providing accurate responses across a wide range of healthcare topics. Their AI systems are designed to support healthcare professionals in diagnosing conditions, offering advice, and providing insights that aid in improving patient care. If you have any further questions or need assistance, feel free to ask!

### **💬 Summary**:

Prashasst is the company behind the AI Doctor you are interacting with. Their expertise lies in developing intelligent AI solutions to assist in medical reasoning and healthcare advice, ensuring accurate and reliable responses to various medical queries.
<｜end▁of▁sentence｜>


In [ ]:
# Initialize FastAPI app
app = FastAPI()



# Define request format
class ChatInput(BaseModel):
    message: str





In [ ]:
# API endpoint for chatbot
@app.post("/chat")
def chat(res: ChatInput):
    response = conversation.predict(input=res.message)
    return {"response": response}

# Set up ngrok tunnel
ngrok.set_auth_token(userdata.get('NGROK'))
ngrok_tunnel = ngrok.connect(8000)
print("Public URL:", ngrok_tunnel.public_url)

# Fix event loop issues in Colab
nest_asyncio.apply()

# Run FastAPI server
uvicorn.run(app, port=8000)


Public URL: https://12d8-34-143-207-67.ngrok-free.app


INFO:     Started server process [2885]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
  

INFO:     49.43.40.15:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     49.43.40.15:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     49.43.40.15:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     49.43.40.15:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     49.43.40.15:0 - "POST /chat HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.11/dist-packages/starlette/applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.11/dist-packages/starlette/middleware/errors.py", line 187, in __call__
    raise exc
  File "/usr/local/lib/python3.11/dist-packages/starlette/middleware/errors.py",